# Enigma machine
Specification from [Codes and Ciphers](http://www.codesandciphers.org.uk/enigma/rotorspec.htm) page.

Example Enigma machines from [Louise Dale](http://enigma.louisedade.co.uk/enigma.html) (full simulation) and [EnigmaCo](http://enigmaco.de/enigma/enigma.html) (good animation of the wheels, but no ring settings).

There's also the nice Enigma simulator for Android by [Franklin Heath](https://franklinheath.co.uk/2012/02/04/our-first-app-published-enigma-simulator/), available on the [Google Play store](https://play.google.com/store/apps/details?id=uk.co.franklinheath.enigmasim&hl=en_GB).

In [333]:
import string

cat = ''.join

In [334]:
wheel_i_spec = 'ekmflgdqvzntowyhxuspaibrcj'
wheel_ii_spec = 'ajdksiruxblhwtmcqgznpyfvoe'
wheel_iii_spec = 'bdfhjlcprtxvznyeiwgakmusqo'
wheel_iv_spec = 'esovpzjayquirhxlnftgkdcmwb'
wheel_v_spec = 'vzbrgityupsdnhlxawmjqofeck'
wheel_vi_spec = 'jpgvoumfyqbenhzrdkasxlictw'
wheel_vii_spec = 'nzjhgrcxmyswboufaivlpekqdt'
wheel_viii_spec = 'fkqhtlxocbjspdzramewniuygv'
beta_wheel_spec = 'leyjvcnixwpbqmdrtakzgfuhos'
gamma_wheel_spec = 'fsokanuerhmbtiycwlqpzxvgjd'

wheel_i_pegs = ['q']
wheel_ii_pegs = ['e']
wheel_iii_pegs = ['v']
wheel_iv_pegs = ['j']
wheel_v_pegs = ['z']
wheel_vi_pegs = ['z', 'm']
wheel_vii_pegs = ['z', 'm']
wheel_viii_pegs = ['z', 'm']

reflector_b_spec = '(ay) (br) (cu) (dh) (eq) (fs) (gl) (ip) (jx) (kn) (mo) (tz) (vw)'
reflector_c_spec = '(af) (bv) (cp) (dj) (ei) (go) (hy) (kr) (lz) (mx) (nw) (tq) (su)'

In [335]:
class LetterTransformer(object):
    def __init__(self, specification, raw_transform=False):
        if raw_transform:
            transform = specification
        else:
            transform = self.parse_specification(specification)
        self.validate_transform(transform)
        self.make_transform_map(transform)
    
    def parse_specification(self, specification):
        return specification
    
    def validate_transform(self, transform):
        """A set of pairs, of from-to"""
        if len(transform) != 26:
            raise ValueError("Transform specification has {} pairs, requires 26".
                format(len(transform)))
        for p in transform:
            if len(p) != 2:
                raise ValueError("Not all mappings in transform "
                    "have two elements")
        if len(set([p[0] for p in transform])) != 26:
            raise ValueError("Transform specification must list 26 origin letters") 
        if len(set([p[1] for p in transform])) != 26:
            raise ValueError("Transform specification must list 26 destination letters") 

    def make_empty_transform(self):
        self.forward_map = [0] * 26
        self.backward_map = [0] * 26
            
    def make_transform_map(self, transform):
        self.make_empty_transform()
        for p in transform:
            self.forward_map[ord(p[0]) - ord('a')] = ord(p[1]) - ord('a')
            self.backward_map[ord(p[1]) - ord('a')] = ord(p[0]) - ord('a')
        return self.forward_map, self.backward_map
    
    def forward(self, letter):
        if letter in string.ascii_lowercase:
            return chr(
                (self.forward_map[(ord(letter) - ord('a')) % 26] + ord('a')))
        else:
            return ''
                
    def backward(self, letter):
        if letter in string.ascii_lowercase:
            return chr(
                (self.backward_map[(ord(letter) - ord('a')) % 26] + ord('a')))
        else:
            return ''

In [336]:
tmap = [('z', 'a')] + [(l, string.ascii_lowercase[i+1]) for i, l in enumerate(string.ascii_lowercase[:-1])]
tmap

[('z', 'a'),
 ('a', 'b'),
 ('b', 'c'),
 ('c', 'd'),
 ('d', 'e'),
 ('e', 'f'),
 ('f', 'g'),
 ('g', 'h'),
 ('h', 'i'),
 ('i', 'j'),
 ('j', 'k'),
 ('k', 'l'),
 ('l', 'm'),
 ('m', 'n'),
 ('n', 'o'),
 ('o', 'p'),
 ('p', 'q'),
 ('q', 'r'),
 ('r', 's'),
 ('s', 't'),
 ('t', 'u'),
 ('u', 'v'),
 ('v', 'w'),
 ('w', 'x'),
 ('x', 'y'),
 ('y', 'z')]

In [337]:
lt = LetterTransformer(tmap)
lt.forward_map, lt.backward_map

([1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  0],
 [25,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24])

In [338]:
cat(lt.forward(l) for l in string.ascii_lowercase)

'bcdefghijklmnopqrstuvwxyza'

In [339]:
cat(lt.backward(l) for l in string.ascii_lowercase)

'zabcdefghijklmnopqrstuvwxy'

In [340]:
class Plugboard(LetterTransformer):
    def parse_specification(self, specification):
        return [tuple(p) for p in specification.split()]
    
    def validate_transform(self, transform):
        """A set of pairs, of from-to"""
        for p in transform:
            if len(p) != 2:
                raise ValueError("Not all mappings in transform"
                    "have two elements")
    
    def make_empty_transform(self):
        self.forward_map = list(range(26))
        self.backward_map = list(range(26))
        
    def make_transform_map(self, transform):
        expanded_transform = transform + [tuple(reversed(p)) for p in transform]
        return super(Plugboard, self).make_transform_map(expanded_transform)

In [341]:
pb = Plugboard([('a', 'z'), ('b', 'y')], raw_transform=True)

In [342]:
cat(pb.forward(l) for l in string.ascii_lowercase)

'zycdefghijklmnopqrstuvwxba'

In [343]:
cat(pb.backward(l) for l in string.ascii_lowercase)

'zycdefghijklmnopqrstuvwxba'

In [344]:
pb = Plugboard('az by')

In [345]:
cat(pb.forward(l) for l in string.ascii_lowercase), cat(pb.backward(l) for l in string.ascii_lowercase)

('zycdefghijklmnopqrstuvwxba', 'zycdefghijklmnopqrstuvwxba')

In [346]:
class Reflector(Plugboard):
    def parse_specification(self, specification):
        return [tuple(p) for p in specification[1:-1].split(') (')]
    
    def validate_transform(self, transform):
        if len(transform) != 13:
            raise ValueError("Reflector specification has {} pairs, requires 13".
                format(len(transform)))
        if len(set([p[0] for p in transform] + 
                    [p[1] for p in transform])) != 26:
            raise ValueError("Reflector specification does not contain 26 letters")
        try:
            super(Reflector, self).validate_transform(transform)
        except ValueError as v:
            raise ValueError("Not all mappings in reflector have two elements")

In [347]:
# reflector_b_text = '(AY) (BR) (CU) (DH) (EQ) (FS) (GL) (IP) (JX) (KN) (MO) (TZ) (VW)'
reflector_b_l = [tuple(p) for p in reflector_b_spec.lower()[1:-1].split(') (')]
reflector_b_l

[('a', 'y'),
 ('b', 'r'),
 ('c', 'u'),
 ('d', 'h'),
 ('e', 'q'),
 ('f', 's'),
 ('g', 'l'),
 ('i', 'p'),
 ('j', 'x'),
 ('k', 'n'),
 ('m', 'o'),
 ('t', 'z'),
 ('v', 'w')]

In [348]:
reflector_b = Reflector(reflector_b_spec, raw_transform=False)

In [349]:
cat(reflector_b.forward(l) for l in string.ascii_lowercase)

'yruhqsldpxngokmiebfzcwvjat'

In [350]:
reflector_c = Reflector(reflector_c_spec)

In [351]:
cat(reflector_c.forward(l) for l in string.ascii_lowercase)

'fvpjiaoyedrzxwgctkuqsbnmhl'

In [352]:
class SimpleWheel(LetterTransformer):
    def __init__(self, transform, position='a', raw_transform=False):
        super(SimpleWheel, self).__init__(transform, raw_transform)
        self.set_position(position)
        
    def parse_specification(self, specification):
        return list(zip(string.ascii_lowercase, specification))
    
    def set_position(self, position):
        self.position = ord(position) - ord('a')
        self.position_l = position
    
    def forward(self, letter):
        if letter in string.ascii_lowercase:
            return chr(
                (self.forward_map[(ord(letter) - ord('a') + self.position) % 26] - 
                    self.position) % 26 + 
                ord('a'))
        else:
            return ''
                
    def backward(self, letter):
        if letter in string.ascii_lowercase:
            return chr(
                (self.backward_map[(ord(letter) - ord('a') + self.position) % 26] - 
                    self.position) % 26 + 
                ord('a'))
        else:
            return ''
        
    def advance(self):
        self.position = (self.position + 1) % 26
        self.position_l = chr(self.position + ord('a'))
        return self.position

In [353]:
rotor_1_transform = list(zip(string.ascii_lowercase, 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'.lower()))
rotor_1_transform

[('a', 'e'),
 ('b', 'k'),
 ('c', 'm'),
 ('d', 'f'),
 ('e', 'l'),
 ('f', 'g'),
 ('g', 'd'),
 ('h', 'q'),
 ('i', 'v'),
 ('j', 'z'),
 ('k', 'n'),
 ('l', 't'),
 ('m', 'o'),
 ('n', 'w'),
 ('o', 'y'),
 ('p', 'h'),
 ('q', 'x'),
 ('r', 'u'),
 ('s', 's'),
 ('t', 'p'),
 ('u', 'a'),
 ('v', 'i'),
 ('w', 'b'),
 ('x', 'r'),
 ('y', 'c'),
 ('z', 'j')]

In [354]:
wheel_1 = SimpleWheel(rotor_1_transform, raw_transform=True)

In [355]:
cat(wheel_1.forward(l) for l in string.ascii_lowercase), cat(wheel_1.backward(l) for l in string.ascii_lowercase)

('ekmflgdqvzntowyhxuspaibrcj', 'uwygadfpvzbeckmthxslrinqoj')

In [356]:
wheel_2 = SimpleWheel(wheel_ii_spec)

In [357]:
cat(wheel_2.forward(l) for l in string.ascii_lowercase), cat(wheel_2.backward(l) for l in string.ascii_lowercase)

('ajdksiruxblhwtmcqgznpyfvoe', 'ajpczwrlfbdkotyuqgenhxmivs')

In [358]:
wheel_3 = SimpleWheel(wheel_iii_spec)
wheel_3.set_position('a')
wheel_3.advance()
cat(wheel_3.forward(l) for l in string.ascii_lowercase)

'cegikboqswuymxdhvfzjltrpna'

In [524]:
class Wheel(SimpleWheel):
    def __init__(self, transform, ring_peg_letters, ring_setting=1, position='a', raw_transform=False):
        self.ring_peg_letters = ring_peg_letters
        self.ring_setting = ring_setting
        super(Wheel, self).__init__(transform, position=position, raw_transform=raw_transform)
        self.set_position(position)
        
    def set_position(self, position):
        super(Wheel, self).set_position(position)
        self.peg_positions = [(ord(p) - ord(position)) % 26  for p in self.ring_peg_letters]
        self.position = (self.position - self.ring_setting + 1) % 26
        
    def advance(self):
        super(Wheel, self).advance()
        self.peg_positions = [(p - 1) % 26 for p in self.peg_positions]
        return self.position

In [525]:
wheel_3 = Wheel(wheel_iii_spec, wheel_iii_pegs, position='b', ring_setting=1)

In [526]:
wheel_3.position, wheel_3.peg_positions

(1, [20])

In [527]:
wheel_6 = Wheel(wheel_vi_spec, wheel_vi_pegs, position='b', ring_setting=3)
wheel_6.position, wheel_6.peg_positions

(25, [24, 11])

In [528]:
for _ in range(27):
    wheel_6.advance()
    print(wheel_6.position, wheel_6.peg_positions)

0 [23, 10]
1 [22, 9]
2 [21, 8]
3 [20, 7]
4 [19, 6]
5 [18, 5]
6 [17, 4]
7 [16, 3]
8 [15, 2]
9 [14, 1]
10 [13, 0]
11 [12, 25]
12 [11, 24]
13 [10, 23]
14 [9, 22]
15 [8, 21]
16 [7, 20]
17 [6, 19]
18 [5, 18]
19 [4, 17]
20 [3, 16]
21 [2, 15]
22 [1, 14]
23 [0, 13]
24 [25, 12]
25 [24, 11]
0 [23, 10]


In [529]:
class Enigma(object):
    def __init__(self, reflector_spec,
                 left_wheel_spec, left_wheel_pegs,
                 middle_wheel_spec, middle_wheel_pegs,
                 right_wheel_spec, right_wheel_pegs,
                 left_ring_setting, middle_ring_setting, right_ring_setting,
                 plugboard_setting):
        self.reflector = Reflector(reflector_spec)
        self.left_wheel = Wheel(left_wheel_spec, left_wheel_pegs, ring_setting=left_ring_setting)
        self.middle_wheel = Wheel(middle_wheel_spec, middle_wheel_pegs, ring_setting=middle_ring_setting)
        self.right_wheel = Wheel(right_wheel_spec, right_wheel_pegs, ring_setting=right_ring_setting)
        self.plugboard = Plugboard(plugboard_setting)
    
    def set_wheels(self, left_wheel_position, middle_wheel_position, right_wheel_position):
        self.left_wheel.set_position(left_wheel_position)
        self.middle_wheel.set_position(middle_wheel_position)
        self.right_wheel.set_position(right_wheel_position)
        
    def lookup(self, letter):
        a = self.plugboard.forward(letter)
        b = self.right_wheel.forward(a)
        c = self.middle_wheel.forward(b)
        d = self.left_wheel.forward(c)
        e = self.reflector.forward(d)
        f = self.left_wheel.backward(e)
        g = self.middle_wheel.backward(f)
        h = self.right_wheel.backward(g)
        i = self.plugboard.backward(h)
        return i
    
    def advance(self):
        advance_middle = False
        advance_left = False
        if 0 in self.right_wheel.peg_positions:
            advance_middle = True
        if 0 in self.middle_wheel.peg_positions:
            advance_left = True
        if 0 in self.middle_wheel.peg_positions and 25 in self.right_wheel.peg_positions:
            advance_middle = True
            advance_left = True
        self.right_wheel.advance()
        if advance_middle: self.middle_wheel.advance()
        if advance_left: self.left_wheel.advance()
            
    def encipher_letter(self, letter):
        self.advance()
        return self.lookup(letter)
    
    def encipher(self, message):
        enciphered = ''
        for letter in message:
            enciphered += self.encipher_letter(letter)
        return enciphered

In [548]:
enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_pegs,
                wheel_ii_spec, wheel_ii_pegs,
                wheel_iii_spec, wheel_iii_pegs,
                1, 1, 1,
                '')

In [549]:
enigma.lookup('a')

'u'

In [550]:
enigma.lookup('u')

'a'

In [551]:
enigma.set_wheels('a', 'a', 'a')
for _ in range(26):
    print(enigma.left_wheel.position_l, enigma.middle_wheel.position_l, enigma.right_wheel.position_l, ';',
          enigma.left_wheel.peg_positions, enigma.middle_wheel.peg_positions, enigma.right_wheel.peg_positions)
    enigma.advance()

a a a ; [16] [4] [21]
a a b ; [16] [4] [20]
a a c ; [16] [4] [19]
a a d ; [16] [4] [18]
a a e ; [16] [4] [17]
a a f ; [16] [4] [16]
a a g ; [16] [4] [15]
a a h ; [16] [4] [14]
a a i ; [16] [4] [13]
a a j ; [16] [4] [12]
a a k ; [16] [4] [11]
a a l ; [16] [4] [10]
a a m ; [16] [4] [9]
a a n ; [16] [4] [8]
a a o ; [16] [4] [7]
a a p ; [16] [4] [6]
a a q ; [16] [4] [5]
a a r ; [16] [4] [4]
a a s ; [16] [4] [3]
a a t ; [16] [4] [2]
a a u ; [16] [4] [1]
a a v ; [16] [4] [0]
a b w ; [16] [3] [25]
a b x ; [16] [3] [24]
a b y ; [16] [3] [23]
a b z ; [16] [3] [22]


In [552]:
enigma.set_wheels('a', 'd', 't')
for _ in range(26):
    print(enigma.left_wheel.position_l, enigma.middle_wheel.position_l, enigma.right_wheel.position_l, ';',
          enigma.left_wheel.peg_positions, enigma.middle_wheel.peg_positions, enigma.right_wheel.peg_positions)
    enigma.advance()

a d t ; [16] [1] [2]
a d u ; [16] [1] [1]
a d v ; [16] [1] [0]
a e w ; [16] [0] [25]
b f x ; [15] [25] [24]
b f y ; [15] [25] [23]
b f z ; [15] [25] [22]
b f a ; [15] [25] [21]
b f b ; [15] [25] [20]
b f c ; [15] [25] [19]
b f d ; [15] [25] [18]
b f e ; [15] [25] [17]
b f f ; [15] [25] [16]
b f g ; [15] [25] [15]
b f h ; [15] [25] [14]
b f i ; [15] [25] [13]
b f j ; [15] [25] [12]
b f k ; [15] [25] [11]
b f l ; [15] [25] [10]
b f m ; [15] [25] [9]
b f n ; [15] [25] [8]
b f o ; [15] [25] [7]
b f p ; [15] [25] [6]
b f q ; [15] [25] [5]
b f r ; [15] [25] [4]
b f s ; [15] [25] [3]


In [553]:
enigma.set_wheels('a', 'a', 'a')
enigma.advance()
enigma.lookup('h')

'i'

In [560]:
enigma.set_wheels('a', 'a', 'a')
ct = enigma.encipher('hellothere')
assert(ct == 'ilbdartydc')
ct

'ilbdartydc'

In [555]:
enigma.set_wheels('a', 'a', 'a')
pt = enigma.encipher(ct)
pt

'hellothere'

In [556]:
enigma.set_wheels('a', 'a', 'a')
enigma.advance()
enigma.lookup('h')

'i'

In [557]:
enigma.set_wheels('a', 'a', 'a')
enigma.advance()
a = enigma.plugboard.forward('h')
b = enigma.right_wheel.forward(a)
c = enigma.middle_wheel.forward(b)
d = enigma.left_wheel.forward(c)
e = enigma.reflector.forward(d)
f = enigma.left_wheel.backward(e)
g = enigma.middle_wheel.backward(f)
h = enigma.right_wheel.backward(g)
i = enigma.plugboard.backward(h)
a, b, c, d, e, f, g, h, i

('h', 'q', 'q', 'x', 'j', 'z', 's', 'i', 'i')

In [558]:
pb = Plugboard('')
pb.forward('h')

'h'

In [561]:
enigma.set_wheels('a', 'd', 't')
ct = enigma.encipher('hellothere')
assert(ct == 'bahxvfrpdc')
ct

'bahxvfrpdc'

In [562]:
enigma.set_wheels('b', 'd', 'q')
ct = enigma.encipher('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
assert(ct == 'kvmmwrlqlqsqpeugjrcxzwpfyiyybwloewrouvkpoztceuwtfjzqwpbqldttsr')
assert(enigma.left_wheel.position_l == 'c')
assert(enigma.middle_wheel.position_l == 'h')
assert(enigma.right_wheel.position_l == 'a')
ct

'kvmmwrlqlqsqpeugjrcxzwpfyiyybwloewrouvkpoztceuwtfjzqwpbqldttsr'

In [563]:
enigma.left_wheel.position_l

'c'

In [564]:
# Setting sheet line 31 from http://www.codesandciphers.org.uk/enigma/enigma3.htm
# Enigma simulation settings are 
# http://enigma.louisedade.co.uk/enigma.html?m3;b;b153;AFTX;AJFE;AU-BG-EY-FP-HL-IN-JZ-OS-QR-TX
w_enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_pegs,
                wheel_v_spec, wheel_v_pegs,
                wheel_iii_spec, wheel_iii_pegs,
                6, 20, 24,
                'ua pf rq so ni ey bg hl tx zj')

In [566]:
w_enigma.set_wheels('j', 'e', 'u')
ct = w_enigma.encipher('hellothere')
print(ct)
assert(ct == 'wzmfiwuntn')
ct

wzmfiwuntn


'wzmfiwuntn'

In [568]:
w_enigma.set_wheels('n', 'y', 'q')
ct = w_enigma.encipher('hellotomweshouldgetbeerorcoffeesoon')
ct

'ayraqvsfkhflwsmkqicvfwawswmiwvvlteb'